In [1]:
!pip install pyodbc


In [2]:
import pandas as pd
import pyodbc

In [3]:
# Step 1: Define your SQL Server connection
conn_str = """
DRIVER={ODBC Driver 17 for SQL Server};
SERVER=PAVAN_KUMAR_CH\\SQLEXPRESS;
DATABASE=HairSalonDB;
Trusted_Connection=yes;
"""
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
print(" Connected to SQL Server")

 Connected to SQL Server


# CSV URL to SQL Table mapping
csv_table_mapping = {
    
    "https://storage.googleapis.com/data_migration_pr1/Future%20Bookings%20(All%20Clients)0.csv": "FutureBookings",
    "https://storage.googleapis.com/data_migration_pr1/No-Show%20Report0.csv": "NoShowReport",
    "https://storage.googleapis.com/data_migration_pr1/Receipt%20Transactions0.csv": "ReceiptTransactions",
    "https://storage.googleapis.com/data_migration_pr1/Service%20Listing0.csv": "ServiceListing",
    "https://storage.googleapis.com/data_migration_pr1/hair_salon_no_show_wrangled_df.csv": "HairSalonNoShowWrangled"
}

In [5]:
# Step 2: CSV URL for ClientCancellations
csv_url = "https://storage.googleapis.com/data_migration_pr1/Client%20Cancellations0.csv"
table_name = "ClientCancellations"

In [6]:
# Step 3: Load CSV using pandas
df = pd.read_csv(csv_url)

# Step 4: Clean NaNs and ensure compatibility
df = df.where(pd.notnull(df), None)

In [7]:
# Step 5: Build dynamic INSERT statement
columns = df.columns.tolist()
placeholders = ", ".join(["?"] * len(columns))
col_names = ", ".join(f"[{col}]" for col in columns)
insert_sql = f"INSERT INTO {table_name} ({col_names}) VALUES ({placeholders})"

In [8]:
# Step 6: Insert row by row
print(f"\n📥 Inserting rows into {table_name}...")
for idx, row in df.iterrows():
    try:
        values = tuple(row[col] for col in columns)
        cursor.execute(insert_sql, values)
    except Exception as e:
        print(f" Insert failed for row {idx + 1}: {row.to_dict()}")


📥 Inserting rows into ClientCancellations...
 Insert failed for row 13: {'CancelDate': '06/30/2018', 'Code': 'TAMA01', 'Service1': 'SHCW', 'Staff': 'KELLY', 'BookingDate': '06/30/2018', 'CanceledBy': 'BECKY', 'Days1': nan}
 Insert failed for row 14: {'CancelDate': '06/30/2018', 'Code': 'TAMA01', 'Service1': 'CBAL', 'Staff': 'KELLY', 'BookingDate': '06/30/2018', 'CanceledBy': 'BECKY', 'Days1': nan}


In [9]:
# URLs and matching table names (excluding ClientCancellations)
csv_table_mapping = {
    "https://storage.googleapis.com/data_migration_pr1/Future%20Bookings%20(All%20Clients)0.csv": "FutureBookings",
    "https://storage.googleapis.com/data_migration_pr1/No-Show%20Report0.csv": "NoShowReport",
    "https://storage.googleapis.com/data_migration_pr1/Receipt%20Transactions0.csv": "ReceiptTransactions",
    "https://storage.googleapis.com/data_migration_pr1/Service%20Listing0.csv": "ServiceListing",
    "https://storage.googleapis.com/data_migration_pr1/hair_salon_no_show_wrangled_df.csv": "HairSalonNoShowWrangled"
}

# Insert data into each table
for url, table_name in csv_table_mapping.items():
    print(f"\n Inserting rows into {table_name}...")
    df = pd.read_csv(url)
    df.fillna("", inplace=True)

    columns = df.columns.tolist()
    placeholders = ', '.join(['?'] * len(columns))
    insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"

    insert_errors = 0
    for i, row in df.iterrows():
        try:
            cursor.execute(insert_sql, tuple(row))
        except Exception as e:
            print(f" Insert failed for row {i + 1}: {row.to_dict()}")
            print(f"   Error: {e}")
            insert_errors += 1

    conn.commit()
    print(f" Insert complete for {table_name} with {insert_errors} errors.")


 Inserting rows into FutureBookings...
 Insert complete for FutureBookings with 0 errors.

 Inserting rows into NoShowReport...
 Insert complete for NoShowReport with 0 errors.

 Inserting rows into ReceiptTransactions...
 Insert complete for ReceiptTransactions with 0 errors.

 Inserting rows into ServiceListing...
 Insert complete for ServiceListing with 0 errors.

 Inserting rows into HairSalonNoShowWrangled...
 Insert complete for HairSalonNoShowWrangled with 0 errors.
